In [2]:
import findspark
findspark.add_jars('/app/postgresql-42.1.4.jar')
findspark.init()

KeyboardInterrupt: 

In [ ]:
from pyspark.sql import SparkSession
spark = (
    SparkSession.builder
    .appName("argentinaDatosAdd")
    .config("spark.driver.memory", "512m")
    .config("spark.driver.cores", "1")
    .config("spark.driver.memory", "15g")
    .config("spark.executor.memory", "512m")
    .config("spark.executor.cores", "1")
    .config("spark.sql.shuffle.partitions", "2")
    .getOrCreate()
)

In [ ]:
from pyspark.sql.functions import *;
from pyspark.sql.types import *;
from scipy.stats import *

In [ ]:
# Create dataframe from table in Postgres DB
df = spark.read.format("jdbc") \
.option("url", "jdbc:postgresql://postgres/human_activity") \
.option("driver", "org.postgresql.Driver") \
.option("dbtable", "human_activity.ha_dataset") \
.option("user", "ha_user") \
.option("password", "hnm4/4c71v1tY") \
.load()

In [1]:
%sql
ha_dataset

UsageError: Line magic function `%sql` not found.


In [ ]:
%sql
WITH
match_total_cards AS(
SELECT league, id_odsp, COUNT(id_odsp) AS match_cards
FROM GAME_EVENTS
WHERE event_type_str = "Red card" OR event_type_str = "Yellow card" OR event_type_str = "Second yellow card"
GROUP BY league, id_odsp
)
SELECT league AS League, ROUND(AVG(match_cards),1) AS AvgCardsPerMatch
FROM match_total_cards
GROUP BY league
ORDER BY league


In [6]:
provRindeCultivos = (
df.groupBy('provincia', 'cultivo', 'campania', 'fecha')
.agg(avg('rendimiento_kgha').alias('rinde_kxha'))
.orderBy(desc('rinde_kxha'))
)

In [7]:
provRindeCultivos \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.prov_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [8]:
sojaBArinde = (
df.filter(df['provincia'] == "BUENOS AIRES")
.filter(df['cultivo'] == 'Soja total')
.select('provincia','rendimiento_kgha', 'campania', 'fecha')
)

In [9]:
sojaBArinde \
    .write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://postgres/agricultura") \
    .option("dbtable", "agricultura.soja_ba_rinde") \
    .option("user", "agricultura") \
    .option("password", pas) \
    .option("driver", "org.postgresql.Driver") \
    .mode('overwrite') \
    .save()

In [10]:
cultivosProvincias = (
df.withColumn("tn_cosechadas",(col("ha_cosechada") * col('rendimiento_kgha'))/1000)
.groupBy('cultivo', 'provincia', 'fecha')
.agg(sum('tn_cosechadas').alias('tn_total'))
.orderBy(desc('tn_total'))
)

In [11]:
cultivosProvincias \
.write \
.format('jdbc') \
.option('url', 'jdbc:postgresql://postgres/agricultura') \
.option('dbtable', 'agricultura.cultivos_provincias') \
.option('user', 'agricultura') \
.option('password', pas) \
.option('driver', 'org.postgresql.Driver') \
.mode('overwrite') \
.save()

In [12]:
spark.stop()